In [1]:
!pip install numpy==1.15
!pip install pandas==0.24.1
!pip install keras==2.1.1

In [0]:
import json 
import os
import collections
from scipy import spatial
import re
import h5py
import argparse
import sys 
import numpy as np 
import pandas as pd
import pickle
import nltk
from nltk.translate.bleu_score import corpus_bleu

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Load necessary files: Here we have loaded file for training
df_final = pickle.load(open('/content/drive/My Drive/code/train_df_final.pkl', 'rb'))
features = pickle.load(open('/content/drive/My Drive/code/image_feature_train.pkl', 'rb'))
word_to_idx = pickle.load(open('/content/drive/My Drive/code/word_to_idx.pkl', 'rb'))
idx_to_word = pickle.load(open('/content/drive/My Drive/code/idx_to_word.pkl', 'rb'))

In [5]:
import keras as k
from keras.layers import Input, Embedding, merge, LSTM, GRU, Dropout, Dense, RepeatVector, BatchNormalization, \
    TimeDistributed, Flatten, Reshape
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras import optimizers
from keras import backend as bk 
bk.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [6]:
PADDING_LEN=17
vocabulary_size=len(word_to_idx)
embed_size=150
question_max_len=PADDING_LEN-1
answer_max_len=PADDING_LEN-1
# Image
k_image_input = Input(shape=(4096,))
k_image_repeat = RepeatVector(n=question_max_len)(k_image_input)

# Question
k_question_input = Input(shape=(question_max_len,), dtype='int32')
k_question_embedded = Embedding(input_dim=vocabulary_size, output_dim=embed_size, input_length=question_max_len)(k_question_input)  
k_question_embedded = Dropout(0.5)(k_question_embedded)

# Merge
k_merged = merge([k_image_repeat, k_question_embedded], mode='concat')  # Merge for layers merge for tensors
k_merged = BatchNormalization()(k_merged)

#Set encoder
k_encoder_outputs, k_state_h, k_state_c  = LSTM(embed_size, return_state=True)(k_merged)
k_encoder_outputs = Dropout(0.5)(k_encoder_outputs)
k_encoder_states = [k_state_h, k_state_c]

# Set up the decoder, using `encoder_states` as initial state.
k_decoder_inputs = Input(shape=(None, vocabulary_size))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
k_decoder_lstm = LSTM(embed_size, return_sequences=True, return_state=True)
k_decoder_outputs, _, _ = k_decoder_lstm(k_decoder_inputs, initial_state=k_encoder_states)

#Final Layer
k_decoder_dense = Dense(vocabulary_size, activation='softmax')
k_decoder_outputs = k_decoder_dense(k_decoder_outputs)

model = Model([k_image_input, k_question_input, k_decoder_inputs], k_decoder_outputs)
print('Model created')
print('Compiling model...')


model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
print('Model compiled')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Model created
Compiling model...
Model compiled


In [0]:
image_input=np.array(features)
question_inputs=np.array(df_final['Q_Encoded'])
question_inputs=np.array([np.array(item) for item in question_inputs])
decoder_inputs =np.array(df_final['A_Encoded'])
decoder_targets =np.array(df_final['A_Encoded'])

In [0]:
m_decoder_inputs = np.zeros(
    (len(decoder_inputs), PADDING_LEN-1, vocabulary_size),
    dtype='float32')
m_decoder_targets= np.zeros(
    (len(decoder_targets), PADDING_LEN-1, vocabulary_size),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(question_inputs, decoder_inputs)):
    for t, char in enumerate(target_text):
        m_decoder_inputs[i, t, char] = 1.
        if t > 0:
            m_decoder_targets[i, t - 1, char] = 1.

In [9]:
model.fit([features,question_inputs, m_decoder_inputs], m_decoder_targets, batch_size=100, epochs=100, validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 3608 samples, validate on 902 samples
Epoch 1/100
3608/3608 [==============================] - 10s 3ms/step - loss: 3.7190 - val_loss: 5.8321
Epoch 2/100
3608/3608 [==============================] - 8s 2ms/step - loss: 2.6712 - val_loss: 3.5934
Epoch 3/100
3608/3608 [==============================] - 8s 2ms/step - loss: 2.5574 - val_loss: 3.1010
Epoch 4/100
3608/3608 [==============================] - 8s 2ms/step - loss: 2.4711 - val_loss: 2.7957
Epoch 5/100
3608/3608 [==============================] - 8s 2ms/step - loss: 2.3854 - val_loss: 2.7043
Epoch 6/100
3608/3608 [==============================] - 8s 2ms/step - loss: 2.3046 - val_loss: 2.6348
Epoch 7/100
3608/3608 [==============================] - 8s 2ms/step - loss: 2.2291 - val_loss: 2.5936
Epoch 8/100
3608/3608 [==============================] - 8s 2ms/step - loss: 2.1528 - val_loss: 2.5149
Ep

In [0]:
encoder_model = Model([k_image_input,k_question_input], k_encoder_states)
decoder_state_input_h = Input(shape=(embed_size,))
decoder_state_input_c = Input(shape=(embed_size,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = k_decoder_lstm(k_decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = k_decoder_dense(decoder_outputs)
decoder_model = Model([k_decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

In [0]:
def decode_sequence(image_features,input_seq):
    # Encode the input as state vectors.
  
    states_value = encoder_model.predict([image_features,input_seq])

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, vocabulary_size))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 1] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx_to_word[sampled_token_index]
        decoded_sentence += sampled_char
        decoded_sentence +=" "

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '2' or
           len(decoded_sentence) > PADDING_LEN-1):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, vocabulary_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
#Open inputs of test dataset
file = open('/content/drive/My Drive/code/image_feature_test.pkl', 'rb')
features_test = pickle.load(file)
file2 = open('/content/drive/My Drive/code/test_df_v_final.pkl', 'rb')
df_test= pickle.load(file2)

In [0]:
test_questions=np.array(df_test['Q_Encoded'])
test_questions=np.array([np.array(item) for item in test_questions])

In [0]:
features_test = pd.DataFrame(features_test)

In [0]:
#use model to predict the answer
predicted_answers_test=[]
for seq_index in range(len(df_test)):
    input_seq = test_questions[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(features_test.iloc[seq_index: seq_index + 1,:], input_seq)
    predicted_answers_test.append(decoded_sentence.split(' '))

In [0]:
predicted_answers_test=[item[0:len(item)-1] for item in predicted_answers_test]

In [0]:
#function used to remove end tokens from the predicted answers
def find_end(item,st):
    for i in range(0,len(item)):
        if item[i]==st:
            return i
    return i+1  
predicted_answers_test=[item[0:find_end(item,'<END>')] for item in predicted_answers_test]

In [0]:
#actual/target answers
test_answers=np.array(df_test['A_parsed'])
test_answers=[item for item in test_answers]

In [19]:
#Final BLEU Score
score_bleu = corpus_bleu(test_answers, predicted_answers_test)
print("BLEU SCORE: ", score_bleu)


BLEU SCORE:  0.37697703778617775


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
